In [ ]:
# Step 1: Check GPU and System Setup
import torch
import os

print("System Check:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Warning: No GPU detected - training will be very slow!")

print(f"\nCurrent directory: {os.getcwd()}")
print(f"Python version: {torch.version.__version__}")

In [ ]:
# Step 2: Install Required Packages
!pip install nibabel scipy pyyaml scikit-learn matplotlib tensorboard tqdm pandas seaborn

print("\nAll requirements installed!")
print("Packages installed:")
print("  - nibabel (NIfTI medical image handling)")
print("  - torch (Deep learning)")
print("  - scipy (Scientific computing)")
print("  - matplotlib (Plotting)")
print("  - tensorboard (Training visualization)")

In [ ]:
# Step 3: Check Dataset Structure
print("Checking uploaded files...")
!ls -la

print("\nMM-WHS Dataset structure:")
!ls -la 'MM-WHS 2017 Dataset'/

print("\nChecking for medical image files:")
!find 'MM-WHS 2017 Dataset' -name "*.nii.gz" | head -10

print("\nFile types in mr_train:")
!find 'MM-WHS 2017 Dataset/mr_train' -name "*.nii.gz" | sed 's/.*\///' | sort

print("\nTotal files per directory:")
for subdir in ['mr_train', 'mr_test', 'ct_train', 'ct_test']:
    print(f"{subdir}:")
    !find 'MM-WHS 2017 Dataset'/'{subdir}' -name "*.nii.gz" | wc -l

In [ ]:
# Step 4: Prepare for Training

# Create necessary directories
!mkdir -p runs/dof_regression
!mkdir -p checkpoints
!mkdir -p synthetic_training_data

print("Created training directories")

# Check if we have actual image files or just labels
image_files = !find 'MM-WHS 2017 Dataset' -name "*image*.nii.gz"
label_files = !find 'MM-WHS 2017 Dataset' -name "*label*.nii.gz"
all_files = !find 'MM-WHS 2017 Dataset/mr_train' -name "*.nii.gz"

print(f"\nDataset Analysis:")
print(f"Image files found: {len(image_files)}")
print(f"Label files found: {len(label_files)}")
print(f"Total mr_train files: {len(all_files)}")

if len(image_files) == 0:
    print("\nWarning: No image files found! The dataset might only contain labels.")
    print("We'll adapt the code to work with available files...")
    
    # Check what files we actually have
    print("\nAvailable files in mr_train:")
    !ls 'MM-WHS 2017 Dataset/mr_train'/ | head -5

In [ ]:
# Step 5: Generate Training Data
print("Generating synthetic training data...")
!python generate_training_data.py
print("\nTraining data generation completed!")

In [ ]:
# Step 6: Train the Model
# Create necessary directories
!mkdir -p runs/dof_regression
!mkdir -p checkpoints
!mkdir -p synthetic_training_data

# Check if synthetic data was generated
print("Checking synthetic training data...")
!ls -la synthetic_training_data/ | head -5

print("\n" + "="*50)
print("Starting model training (this will take 30-60 minutes)...")

# Start training
!python dof_regressor_model.py --metadata-dir synthetic_training_data
print("\nModel training completed!")

In [ ]:
# Step 7: Check Results
print("Files created during training:")
!ls -la checkpoints/
print("\nTraining logs:")
!ls -la runs/dof_regression/
print("\nModel file size:")
!du -h checkpoints/best_checkpoint.pth 2>/dev/null || echo "Model file not found"

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("Your trained model: checkpoints/best_checkpoint.pth") 
print("Training logs: runs/dof_regression/")
print("Demo script: demo.py")
print("\nDownload these files to deliver to your client!")

In [ ]:
# Step 8: Run Demo
!python demo.py

In [ ]:
# Step 9: Download Trained Model
from google.colab import files

print("Downloading trained model...")
files.download('checkpoints/best_checkpoint.pth')
print("Download complete!")